# <font color='blue'>Formação Cientista de Dados
Projeto com Feedback 4
    </font>

## Eduardo Frigini de Jesus

## Prevendo Customer Churn em Operadoras de Telecom

In [1]:
# Importando os módulos
import pandas as pd                 
import matplotlib.pyplot as plt    
import numpy as np         
import os
%matplotlib inline             

import os.path
import pickle
import seaborn as sns
import math
import gc

from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

# import xlrd
# import xlsxwriter

# from IPython.display import display
# from ipywidgets import widgets
# from IPython.display import display, clear_output

In [ ]:
# Carregando o dataset
df_treino = pd.read_csv("projeto4_telecom_treino.csv")
df_teste = pd.read_csv("projeto4_telecom_teste.csv")

In [ ]:
df_treino = df_treino.set_index('Key')
df_teste = df_teste.set_index('Key')

In [ ]:
type(df_treino) 

In [ ]:
type(df_teste)

In [ ]:
# Verificando o formato dos dados
df_treino.shape

In [ ]:
df_teste.shape

In [ ]:
# Verificando as primeiras linhas do dataset
df_treino.head(5)

In [ ]:
df_teste.head(5)

In [ ]:
# Verificando se existem valores nulos
df_treino.isnull().values.any()

In [ ]:
df_treino.isnull().values.any()

In [ ]:
# Verificando as últimas linhas do dataset
df_treino.tail(5)

In [ ]:
df_teste.tail(5)

# Ajustando os datasets, substituindo yes e no por 1 e 0

In [ ]:
df_treino.dtypes

In [ ]:
df_teste.dtypes

In [ ]:
# Definindo as classes. Substituindo o yes por 1 e no por 0
change_map = {'yes' : 1, 'no' : 0}

In [ ]:
# Aplicando o mapeamento ao dataset de treino
df_treino['churn'] = df_treino['churn'].map(change_map)
df_treino['international_plan'] = df_treino['international_plan'].map(change_map)
df_treino['voice_mail_plan'] = df_treino['voice_mail_plan'].map(change_map)

In [ ]:
# Aplicando o mapeamento ao dataset de teste
df_teste['churn'] = df_teste['churn'].map(change_map)
df_teste['international_plan'] = df_teste['international_plan'].map(change_map)
df_teste['voice_mail_plan'] = df_teste['voice_mail_plan'].map(change_map)

In [ ]:
df_treino.tail(5)

In [ ]:
df_teste.tail(5)

# Verificando e Balanceando os dados de treino e teste

In [ ]:
# Verificando como os dados estão distribuídos, no treino
num_true = len(df_treino.loc[df_treino['churn'] == True])
num_false = len(df_treino.loc[df_treino['churn'] == False])
print("Número de Casos Verdadeiros: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Casos Falsos     : {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

In [ ]:
# Balanceando os dados de treino
df_treino_true = (df_treino.loc[df_treino['churn']==True])
df_treino_false = (df_treino.loc[df_treino['churn']==False]).sample(483)
df_treino_balance = pd.concat([df_treino_true, df_treino_false])
df_treino_balance.head(5)

In [ ]:
# Verificando como os dados estão distribuídos, no treino
num_true = len(df_treino_balance.loc[df_treino_balance['churn'] == True])
num_false = len(df_treino_balance.loc[df_treino_balance['churn'] == False])
print("Número de Casos Verdadeiros: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Casos Falsos     : {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

In [ ]:
# Verificando como os dados estão distribuídos, no teste
num_true = len(df_teste.loc[df_teste['churn'] == True])
num_false = len(df_teste.loc[df_teste['churn'] == False])
print("Número de Casos Verdadeiros: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Casos Falsos     : {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

In [ ]:
# Balanceando os dados de teste
df_teste_true = (df_teste.loc[df_teste['churn']==True])
df_teste_false = (df_teste.loc[df_teste['churn']==False]).sample(224)
df_teste_balance = pd.concat([df_teste_true, df_teste_false])
df_teste_balance.head(5)

In [ ]:
# Verificando como os dados estão distribuídos, no teste
num_true = len(df_teste_balance.loc[df_teste_balance['churn'] == True])
num_false = len(df_teste_balance.loc[df_teste_balance['churn'] == False])
print("Número de Casos Verdadeiros: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Casos Falsos     : {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

In [ ]:
# Identificando a correlação entre as variáveis
# Correlação não implica causalidade
def plot_corr(df, size=10):
    corr = df.corr()    
    fig, ax = plt.subplots(figsize = (size, size))
    ax.matshow(corr)  
    plt.xticks(range(len(corr.columns)), corr.columns) 
    plt.yticks(range(len(corr.columns)), corr.columns)  

In [ ]:
# Criando o gráfico
plot_corr(df_treino_balance)

In [ ]:
# Visualizando a correlação em tabela
# Coeficiente de correlação: 
# +1  = forte correlação positiva
# 0   = não há correlação
# -1  = forte correlação negativa
df_treino_balance.corr()

In [ ]:
# Decidi juntar os dois dataframes e separar novamente
df = pd.concat([df_treino_balance, df_teste_balance])

In [ ]:
# Verificando como os dados estão distribuídos
num_true = len(df.loc[df['churn'] == True])
num_false = len(df.loc[df['churn'] == False])
print("Número de Casos Verdadeiros: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Casos Falsos     : {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

## Spliting

70% para dados de treino e 30% para dados de teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Seleção de variáveis preditoras (Feature Selection)
atributos = ['international_plan', 'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes', 'total_day_charge', 'total_eve_minutes', 'total_eve_charge', 'number_customer_service_calls']

In [ ]:
# Variável a ser prevista
atrib_prev = ['churn']

In [ ]:
# Criando objetos
X = df[atributos].values
Y = df[atrib_prev].values

In [ ]:
X

In [ ]:
Y

In [ ]:
# Definindo a taxa de split
split_test_size = 0.30

In [ ]:
# Criando dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = split_test_size, random_state = 42)

In [ ]:
# Imprimindo os resultados
print("{0:0.2f}% nos dados de treino".format((len(X_treino)/len(df.index)) * 100))
print("{0:0.2f}% nos dados de teste".format((len(X_teste)/len(df.index)) * 100))

In [ ]:
X_treino

## Verificando o Split

In [ ]:
print("Original True : {0} ({1:0.2f}%)".format(len(df.loc[df['churn'] == 1]), 
                                               (len(df.loc[df['churn'] ==1])/len(df.index) * 100)))

print("Original False : {0} ({1:0.2f}%)".format(len(df.loc[df['churn'] == 0]), 
                                               (len(df.loc[df['churn'] == 0])/len(df.index) * 100)))
print("")
print("Training True : {0} ({1:0.2f}%)".format(len(Y_treino[Y_treino[:] == 1]), 
                                               (len(Y_treino[Y_treino[:] == 1])/len(Y_treino) * 100)))

print("Training False : {0} ({1:0.2f}%)".format(len(Y_treino[Y_treino[:] == 0]), 
                                               (len(Y_treino[Y_treino[:] == 0])/len(Y_treino) * 100)))
print("")
print("Test True : {0} ({1:0.2f}%)".format(len(Y_teste[Y_teste[:] == 1]), 
                                               (len(Y_teste[Y_teste[:] == 1])/len(Y_teste) * 100)))

print("Test False : {0} ({1:0.2f}%)".format(len(Y_teste[Y_teste[:] == 0]), 
                                               (len(Y_teste[Y_teste[:] == 0])/len(Y_teste) * 100)))

## Valores Missing Ocultos



In [ ]:
# Verificando se existem valores nulos
df.isnull().values.any()

## Construindo e treinando o modelo

In [ ]:
# Utilizando um classificador Naive Bayes
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Criando o modelo preditivo
modelo_v1 = GaussianNB()

In [ ]:
# Treinando o modelo
modelo_v1.fit(X_treino, Y_treino.ravel())

## Verificando a exatidão no modelo nos dados de treino

In [ ]:
from sklearn import metrics

In [ ]:
nb_predict_train = modelo_v1.predict(X_treino)

In [ ]:
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, nb_predict_train)))
print()

## Verificando a exatidão no modelo nos dados de teste

In [ ]:
nb_predict_test = modelo_v1.predict(X_teste)

In [ ]:
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, nb_predict_test)))
print()

## Métricas

In [ ]:
# Criando uma Confusion Matrix
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(Y_teste, nb_predict_test, labels = [1, 0])))
print("")

print("Classification Report")
print(metrics.classification_report(Y_teste, nb_predict_test, labels = [1, 0]))

# Otimizando o modelo com RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
modelo_v2 = RandomForestClassifier(random_state = 42)
modelo_v2.fit(X_treino, Y_treino.ravel())

In [ ]:
# Verificando os dados de treino
rf_predict_train = modelo_v2.predict(X_treino)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, rf_predict_train)))

In [ ]:
# Verificando nos dados de teste
rf_predict_test = modelo_v2.predict(X_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, rf_predict_test)))
print()

In [ ]:
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(Y_teste, rf_predict_test, labels = [1, 0])))
print("")

print("Classification Report")
print(metrics.classification_report(Y_teste, rf_predict_test, labels = [1, 0]))

## Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Terceira versão do modelo usando Regressão Logística
modelo_v3 = LogisticRegression(C = 0.7, random_state = 42)
modelo_v3.fit(X_treino, Y_treino.ravel())
lr_predict_test = modelo_v3.predict(X_teste)

In [ ]:
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, lr_predict_test)))
print()
print("Classification Report")
print(metrics.classification_report(Y_teste, lr_predict_test, labels = [1, 0]))

In [ ]:
### Resumindo
## Exatidão nos dados de teste

# Modelo usando algoritmo Naive Bayes         = 0.8306
# Modelo usando algoritmo Random Forest       = 0.8612
# Modelo usando algoritmo Regressão Logística = 0.7906

## Fazendo Previsões Com o Modelo Treinado
## Random Forest foi o melhor modelo

In [ ]:
import pickle

In [ ]:
# Salvando o modelo para usar mais tarde
filename = 'modelo_treinado_v2.sav'
pickle.dump(modelo_v2, open(filename, 'wb'))

In [ ]:
X_teste

In [ ]:
# Carregando o modelo e fazendo previsão com novos conjuntos de dados 
# (X_teste, Y_teste devem ser novos conjuntos de dados preparados com o procedimento de limpeza e transformação adequados)
loaded_model = pickle.load(open(filename, 'rb'))
resultado1 = loaded_model.predict(X_teste[10].reshape(1, -1))
resultado2 = loaded_model.predict(X_teste[40].reshape(1, -1))
print(resultado1)
print(resultado2)

# Fim